In [ ]:
from selenium import webdriver #1st import: Allows you to launch/initialise a browser.
from selenium.webdriver.common.by import By  #2nd import: Allows you to search for things using specific parameters.
from selenium.webdriver.support.ui import WebDriverWait #3rd import: Allows you to wait for a page to load.
from selenium.webdriver.support import expected_conditions as EC #4th import: Specify what you are looking for on a specific page in order to determine that the webpage has loaded.
from selenium.common.exceptions import TimeoutException #5th import: Handling a timeout situation.
import time
from selenium.webdriver.common.keys import Keys
import traceback

In [ ]:
#First we start by adding the incognito argument to our webdriver.
option = webdriver.ChromeOptions()
option.add_argument("--incognito")
#option.add_argument("--headless") #this option runs the scraping job behind the scenes. However, I wasn't getting results when I activted this. ill re-visit later
option.add_experimental_option("excludeSwitches", ["enable-automation"])
option.add_experimental_option('useAutomationExtension', False) #disables webbrowser notification stating that it is being controlled by automated software

In [ ]:
driver = webdriver.Chrome(executable_path="C:/Users/Owner/Downloads/chromedriver.exe", options=option)

In [ ]:
driver.get('https://locations.chipotle.com')
time.sleep(1)

In [ ]:
stores_states=driver.find_elements(By.XPATH, "//li[@class = 'Directory-listItem']")

In [ ]:
len(stores_states)

In [ ]:
#FINAL code compliing

###Final Code Compliation

#driver.get('https://locations.chipotle.com') - use when testing code to reset starting point to chipotle locations page

#Creating dictonary to capture data
chipotle_stores_dict={'Store_Name':[], 'Store_Street_Address':[], 'Store_State':[], 'Store_State_Code':[], 'Store_City':[],'Store_Zipcode':[], 'Store_Phone_Number':[]}

#Capturing elements for States Listed in Locations Page
stores_states=driver.find_elements(By.XPATH, "//li[@class = 'Directory-listItem']")

#Locations page loop step 1: figuring out how to direct loop based on store count of state: success
for state in stores_states:# -- place a range in plce of 'stores_states' to test for a range
    #Assigning number to pass into element to go through every state listed in the locations page or range listed for testing
    n= stores_states.index(state)+1 #n=stores_states.index(state)+1
    
    #Getting State Name
    store_state_element=driver.find_element(By.XPATH, '//*[@id="main"]/div/section/div[2]/ul/li[{}]/a/span'.format(n))
    store_state=store_state_element.text
    
    #Get Count of Stores in State
    store_state_store_count_element=driver.find_element(By.XPATH, '//*[@id="main"]/div/section/div[2]/ul/li[{}]/a'.format(n))
    store_state_store_count=store_state_store_count_element.get_attribute("data-count").replace("(","").replace(")","")
    print("Extracting data for {} stores in {} State".format(store_state_store_count,store_state))
    
    #Obtain element to store listed in state link
    store_state_link_element=driver.find_element(By.XPATH, '//*[@id="main"]/div/section/div[2]/ul/li[{}]/a'.format(n))
    #Click to navigate to store page
    driver.execute_script("arguments[0].click();", store_state_link_element)
    
    time.sleep(.2)
    
    
    
    #Code below starts navigation pattern dependent on count of stores in the state. If count is one, code goes extract data. If Count is not 1, code navigates to view stores by City
    
    if store_state_store_count == "1":
       # print("{} has one store".format(store_state))
    
        %run extract_chipotle_location_data_fields.ipynb


    #Start of code for states that have more than one store in any given state
    
    else:
        
        #Determining Number of Cities listed in the locations page of State
        stores_cities=driver.find_elements(By.XPATH, "//li[@class = 'Directory-listItem']")

        for city in stores_cities:
            #Assigning number to pass into element to go through every state listed in the locations page
            n=stores_cities.index(city)+1

            #Getting City Name
            store_city_element=driver.find_element(By.XPATH, '//*[@id="main"]/div/section/div[2]/ul/li[{}]/a/span'.format(n))
            store_city=store_city_element.text

            #Get Count of Stores in City
            city_store_count_element=driver.find_element(By.XPATH, '//*[@id="main"]/div/section/div[2]/ul/li[{}]/a'.format(n))
            city_store_count=city_store_count_element.get_attribute("data-count").replace("(","").replace(")","")
            #print(city_store_count)
            
            

            #Code below starts navigation pattern dependent on count of stores in the state. If count is one, code goes extract data. If Count is not 1, code navigates to view stores by City
            if city_store_count == "1":
                #print("{}, {} has one store".format(store_city,store_state))
                
                #Obtain element to store listed in state link
                
                city_store_link_element=driver.find_element(By.XPATH, '//*[@id="main"]/div/section/div[2]/ul/li[{}]/a'.format(n))
                #Click to navigate to store page
                driver.execute_script("arguments[0].click();", city_store_link_element)
                
                time.sleep(.2)

                %run extract_chipotle_location_data_fields.ipynb

                #retuning to page listing stores for particular state  
                driver.execute_script("window.history.go(-1)")
                time.sleep(.5)
            
            else:
                
                #Obtain element to store listed in state link
                store_link_element=driver.find_element(By.XPATH, '//*[@id="main"]/div/section/div[2]/ul/li[{}]/a'.format(n))
                
                #Click to navigate to store page
                driver.execute_script("arguments[0].click();", store_link_element)
                
                time.sleep(.2)


                #Getting elements for store links
                store_links=driver.find_elements(By.XPATH,"//a[@class = 'Teaser-titleLink']")

                for store_link in store_links:
                    n=store_links.index(store_link)+1
                    
                    #Obtain element to store listed in state link
                    store_link_element=driver.find_element(By.XPATH, '//*[@id="main"]/div/section/div[2]/ul/li[{}]/article/h2/a'.format(n))
                    
                    #Click to navigate to store page
                    driver.execute_script("arguments[0].click();", store_link_element)

                    %run extract_chipotle_location_data_fields.ipynb
                    
                    #Retruning to page listing links to various stores in city
                    driver.execute_script("window.history.go(-1)")
                    time.sleep(.5)
                    
                #retuning to page listing cities for store in aparticular state   
                driver.execute_script("window.history.go(-1)")
                time.sleep(.5)
            
            
    #Returning to main state locations page            
    driver.execute_script("window.history.go(-1)")
    time.sleep(.5)

driver.quit()
print("Extraction Complete!")
